# Companies House matching exercise

### Exercise
- Develop a model to match "informal" British company names to official company records from Companies House
- The model may consider company name, location, and any relevant information
- An extract of all currently registered companies can be downloaded from here: http://download.companieshouse.gov.uk/en_output.html

### Overall summary of approach:
Prepare data:
- Define a cleaning strategy which standardises punctuation, white space, capitalisation, and common company "stop words" (like LIMITED/LTD)
- Clean all companies house company names using this function
To find a match:
- If some level of postcode is reliably available (area/district/sector/postcode), the filter companies house data down to companies with exactly matching geography
- Define a similarity/distance metric to compare two strings (various packages to explore)
- Within the (filtered) set of companies, find the closest (or n closest) matches using that metric.

In Option 1, our similarity metric is 1 for exact matches and 0 otherwise (ie just a join)

In Option 2, our similarity metric is Jaro-Winkler distance - ie a typical fuzzy matching algorithm

In Option 3, our similarity metric is cosine similarity between vectorised embeddings of the strings based on n-grams. This allows a fast k-nearest neighbour approach.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# Postcode helper functions written for this exercise
from postcodes import clean_postcode, postcode_breakdown, detect_postcode_type

# Fuzzy matching
import recordlinkage

# Nearest neighbour
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

import pickle as p

### Test data
- No test data (ie sample "informal" company names / addresses) has been provided, so I have generated my own.
- The appropriate model to use would depend on the nature of the company data available for matching.
- To explore a few technical ideas, I will assume that we have been provided two fields for matching:
  - CompanyName: variations on "informal" company names, such as shortenings, typos, extra words etc
  - PostCode: a mixture of full postcodes, areas, districts
- I have created such a file myself, using a handful of companies

In [2]:
df_test = pd.read_csv(
    'data/test_companies.csv'
)

In [3]:
df_test[
    ['postcode_area', 'postcode_district', 'postcode_sector', 'postcode_full']
] = postcode_breakdown(df_test['PostCode_actual'])

# Check 10 random rows
df_test.iloc[np.random.randint(0, df_test.shape[0], 10)]

,CompanyNumber_actual,CompanyName_actual,PostCode_actual,CompanyName,TestType,postcode_area,postcode_district,postcode_sector,postcode_full
29,7886430,BOUGHT BY MANY LTD,EC1R 5BD,bouht bym any,typo,EC,EC1R,EC1R 5,EC1R 5BD
5,10207490,MUNICH RE DIGITAL PARTNERS LIMITED,EC3M 5BN,Munch Ree Digitl Pratner,typo,EC,EC3M,EC3M 5,EC3M 5BN
10,FC035970,WILLIS TOWERS WATSON,EC3M 7DQ,Towers Watson Willis,word order,EC,EC3M,EC3M 7,EC3M 7DQ
26,7886430,BOUGHT BY MANY LTD,EC1R 5BD,boughtbymany,white space,EC,EC1R,EC1R 5,EC1R 5BD
33,8331958,DEE BEE MOT LIMITED,SW19 2BY,Dee Bee motors,typo,SW,SW19,SW19 2,SW19 2BY
31,8331958,DEE BEE MOT LIMITED,SW19 2BY,Dee Bee Tyres and MOT,extra words,SW,SW19,SW19 2,SW19 2BY
39,11679582,BRAYNE & SONS BUILDERS LIMITED,SL6 2ED,brayneandsons builders,white space,SL,SL6,SL6 2,SL6 2ED
2,10207490,MUNICH RE DIGITAL PARTNERS LIMITED,EC3M 5BN,Munich Re Digital Partners,normal,EC,EC3M,EC3M 5,EC3M 5BN
28,7886430,BOUGHT BY MANY LTD,EC1R 5BD,bought by many (bbm),extra words,EC,EC1R,EC1R 5,EC1R 5BD
7,FC035970,WILLIS TOWERS WATSON,EC3M 7DQ,Willis Towers Watson,normal,EC,EC3M,EC3M 7,EC3M 7DQ


### Companies House data
**Data source**: http://download.companieshouse.gov.uk/en_output.html

For this exercise, I am just using the company name and postcode for matching.
Some records won't have valid postcodes, eg if addresses are non-UK. They won't be included for any postcode-based matching.


In [4]:
%%time

## Data source: http://download.companieshouse.gov.uk/en_output.html

## Optional code to inspect file:
# df_sample = pd.read_csv(
#     'data/BasicCompanyDataAsOneFile-2020-11-01.csv',
#     nrows=100
# )

# df_sample.head()

columns_to_load = [
    'CompanyName',
    'CompanyNumber',
    'RegAddress.PostCode',
]

df = pd.read_csv(
    'data/BasicCompanyDataAsOneFile-2020-11-01.csv',
    usecols = columns_to_load,
    skipinitialspace=True,
)

print(f'Shape: {df.shape}')

Shape: (4785590, 3)
CPU times: user 13.4 s, sys: 2.09 s, total: 15.5 s
Wall time: 16.3 s


### Data cleaning

Company names in the Companies House data will be cleaned by:
- Consistently using "AND" instead of "&"
- Removing apostrophes (so that "ABC'S" becomes "ABCS")
- Replacing other punctuation with a space (so that "ABC.DEF" becomes "ABC DEF")
- Dropping common company "stop words" (LIMITED, LTD, PLC etc) that are unlikely to be in an "informal" company name
- Trimming excess white space

We also use the postcode helper functions to get a breadkown of the full postcode

In [5]:
def clean_company_name(x, ignore_words=[]):
    # Convert x to a pandas series
    x = pd.Series(np.atleast_1d(x))
    
    # Swap & for AND
    x = x.str.replace('[\&]',' AND ')
    
    # Remove ' (avoids splitting words)
    x = x.str.replace('[\']','')
    
    # Swap other punctuation with a space (keeps words separate)
    x = x.str.replace('[^\w\s]',' ')
    
    # Build regex pattern to remove ignore_words
    pattern = r'\b(?:{})\b'.format('|'.join(ignore_words))
    
    # Remove ignore_words
    x = x.str.replace(pattern, '')
    
    # Remove multiple spaces and trim white space
    x = x.str.replace(' +',' ').str.strip().str.upper()
    
    return x

In [6]:
%%time
ignore_words = ['LIMITED', 'LTD', 'PLC', 'LLP', 'GROUP', 'CO', 'COMPANY', 'UK']

df['CompanyName_clean'] = clean_company_name(df['CompanyName'], ignore_words)

df[['postcode_area', 'postcode_district', 'postcode_sector', 'postcode_full']] = postcode_breakdown(df['RegAddress.PostCode'])

CPU times: user 53.8 s, sys: 3.63 s, total: 57.4 s
Wall time: 58.3 s


In [7]:
df.head()

,CompanyName,CompanyNumber,RegAddress.PostCode,CompanyName_clean,postcode_area,postcode_district,postcode_sector,postcode_full
0,! LIMITED,12778855,S35 2PH,,S,S35,S35 2,S35 2PH
1,! LTD,08209948,LS10 2RU,,LS,LS10,LS10 2,LS10 2RU
2,!? LTD,11399177,SK6 3DY,,SK,SK6,SK6 3,SK6 3DY
3,!BIG IMPACT GRAPHICS LIMITED,11743365,EC1V 9LT,BIG IMPACT GRAPHICS,EC,EC1V,EC1V 9,EC1V 9LT
4,!L PRODUCTIONS LIMITED,12402527,E4 8EJ,L PRODUCTIONS,E,E4,E4 8,E4 8EJ


Apply the same cleaning to the test data 

In [8]:
df_test['CompanyName_clean'] = clean_company_name(df_test['CompanyName'], ignore_words)

### Option 1: Simple join
The company name cleaning may be enough to match some companies from the test set. Let's see how well it does.

In [9]:
def match_companies_simple_join(x):
    # Convert x to a pandas series
    x = pd.Series(np.atleast_1d(x))
    
    # Build a data frame with the cleaned name
    df_results = pd.DataFrame(
        {
            'id': range(len(x)),
            'CompanyName_tests': x,
            'CompanyName_clean': clean_company_name(x, ignore_words),
            
        }
    )
    
    df_results = df_results.merge(df, how='left', on='CompanyName_clean')
    
    df_results = df_results.groupby('id').first()
    
    return df_results

# test the function
match_companies_simple_join('hiscox')

,CompanyName_tests,CompanyName_clean,CompanyName,CompanyNumber,RegAddress.PostCode,postcode_area,postcode_district,postcode_sector,postcode_full
id,,,,,,,,,
0,hiscox,HISCOX,HISCOX PLC,02837811,EC3A 6HX,EC,EC3A,EC3A 6,EC3A 6HX


In [10]:
%%time 
df_results = match_companies_simple_join(df_test['CompanyName'])

CPU times: user 6.44 s, sys: 5.68 s, total: 12.1 s
Wall time: 14.2 s


In [11]:
## Let's check how we did:
df_test['simple_join_correct'] = (df_results['CompanyName'] == df_test['CompanyName_actual'])

print(f"{df_test['simple_join_correct'].sum()} out of {len(df_test)} correct")

df_test.pivot_table(
    values='CompanyName',
    index='simple_join_correct',
    columns='TestType',
    aggfunc='count'
)

10 out of 40 correct


TestType,extra words,normal,shortened,typo,white space,word order
simple_join_correct,,,,,,
False,6.0,NaN,8.0,7.0,5.0,4.0
True,1.0,9.0,NaN,NaN,NaN,NaN


Just using a simple join got our 'normal' test cases correct, and one of the 'extra words' (presumably where the extra word was dropped in the cleaning), but did poorly in most other use cases

### Option 2: Fuzzy Matching package

Try using the `recordlinkage` package to implement common fuzzy matching algorithms

Note that this is an approach I have used previously, and originally found here: https://pbpython.com/record-linking.html

This approach uses the Jaro-Winkler distance between strings, which roughly quantifies the number of edits to transform one string into another.
https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance

In [12]:
%%time
indexer = recordlinkage.Index()
indexer.full()

candidates = indexer.index(df_test, df)
print(f'Comparisons: {len(candidates)}')

Comparisons: 191423600
CPU times: user 2.31 s, sys: 906 ms, total: 3.22 s
Wall time: 3.36 s


A full indexer would need to do 191,423,600 pairwise comparisons (ie every test company to every companies house record) , which would be very slow.
We can use the postcode to pre-filter the number of comparisons and speed things up.

In [13]:
%%time
indexer = recordlinkage.Index()
indexer.block(left_on='postcode_area', right_on='postcode_area')

candidates = indexer.index(df_test, df)
print(f'Comparisons: {len(candidates)}')

compare = recordlinkage.Compare()

compare.string(
    left_on='CompanyName_clean',
    right_on='CompanyName_clean',
    method='jarowinkler',
)

features = compare.compute(
    candidates, 
    df_test,
    df
)

# Find the best match score per test case
features.reset_index(inplace=True)
features.columns = ['test', 'match', 'score']

features.sort_values(by=['test', 'score'], ascending = False, inplace=True)

best_match = features.groupby('test').first()[['match', 'score']]

# Join the companies house data for the best match
best_match = best_match.merge(
    df,
    how='left',
    left_on='match',
    right_index=True
)

## Let's check how we did:
df_test['recordlinkage_postcode_area_correct'] = (best_match['CompanyName'] == df_test['CompanyName_actual'])

print(f"{df_test['recordlinkage_postcode_area_correct'].sum()} out of {len(df_test)} correct")

df_test.pivot_table(
    values='CompanyName',
    index='recordlinkage_postcode_area_correct',
    columns='TestType',
    aggfunc='count'
)

Comparisons: 4573358
26 out of 40 correct
CPU times: user 2min 59s, sys: 8.83 s, total: 3min 7s
Wall time: 3min 12s


TestType,extra words,normal,shortened,typo,white space,word order
recordlinkage_postcode_area_correct,,,,,,
False,1,1,7,1,1,3
True,6,8,1,6,4,1


In the method above, pre-filtering to matching postcode areas reduces the number of comparisons required to 4,573,358. This should be a fair amount quicker and more accurate than the full indexer.

However, at 3 minutes for 40 records it isn't very quick so may not be appropriate at scale or where prediction speed is important.

Note that to do the 190 million comparisons without postcode filtering would have take 3 * (191423600 / 4573358) / 60 = ~2 hours for 40 records.

### Option 2b: Fuzzy match will full postcode block
We could speed it up and get more accuracy by using a more granular index blocker (ie full postcode instead of postcode area).

In practice, this may not be possible as depending on the quality and availability of a postcode field.

In [14]:
%%time
indexer = recordlinkage.Index()
indexer.block(left_on='postcode_full', right_on='postcode_full')

candidates = indexer.index(df_test, df)
print(f'Comparisons: {len(candidates)}')

compare = recordlinkage.Compare()

compare.string(
    left_on='CompanyName_clean',
    right_on='CompanyName_clean',
    method='jarowinkler',
)

features = compare.compute(
    candidates, 
    df_test,
    df
)

# Find the best match score per test case
features.reset_index(inplace=True)
features.columns = ['test', 'match', 'score']

features.sort_values(by=['test', 'score'], ascending = False, inplace=True)

best_match = features.groupby('test').first()[['match', 'score']]

# Join the companies house data for the best match
best_match = best_match.merge(
    df,
    how='left',
    left_on='match',
    right_index=True
)

## Check how we did:
df_test['recordlinkage_full_postcode_correct'] = (best_match['CompanyName'] == df_test['CompanyName_actual'])

print(f"{df_test['recordlinkage_full_postcode_correct'].sum()} out of {len(df_test)} correct")

df_test.pivot_table(
    values='CompanyName',
    index='recordlinkage_full_postcode_correct',
    columns='TestType',
    aggfunc='count'
)

Comparisons: 2263
30 out of 40 correct
CPU times: user 2.99 s, sys: 946 ms, total: 3.93 s
Wall time: 4.3 s


TestType,extra words,normal,shortened,typo,white space,word order
recordlinkage_full_postcode_correct,,,,,,
False,1,1,4,1,1,2
True,6,8,4,6,4,2


Pre-filtering to match on full postcode reduces the number of comparisons required to 2,263, so the model is very quick.

It's still getting 10 of the test cases wrong. Where is the model going wrong?

In [15]:
errors = df_test['recordlinkage_full_postcode_correct'] == False
pd.DataFrame({
    'CompanyName_test': df_test['CompanyName'],
    'CompanyName_actual': df_test['CompanyName_actual'],
    'CompanyName_guess':best_match['CompanyName']
})[errors]

,CompanyName_test,CompanyName_actual,CompanyName_guess
7,Willis Towers Watson,WILLIS TOWERS WATSON,WILLIS TOWERS WATSON UK HOLDINGS LIMITED
8,WillisTowersWatson,WILLIS TOWERS WATSON,WILLIS TOWERS WATSON UK HOLDINGS LIMITED
9,Towers Watson,WILLIS TOWERS WATSON,WILLIS TOWERS WATSON UK HOLDINGS LIMITED
10,Towers Watson Willis,WILLIS TOWERS WATSON,WILLIS TOWERS WATSON UK HOLDINGS 2 LIMITED
11,Willis Tower Watsons,WILLIS TOWERS WATSON,WILLIS TOWERS WATSON UK HOLDINGS LIMITED
13,Hudson & Thames,THAMES & HUDSON LIMITED,THAMES & HUDSON DIGITAL LTD
15,Thames & Hudson Publishing,THAMES & HUDSON LIMITED,THAMES & HUDSON PUBLISHING LIMITED
18,faber,FABER AND FABER LIMITED,FABER MUSIC LIMITED
20,random house,PENGUIN RANDOM HOUSE LIMITED,RANDOM HOUSE UK VENTURES LIMITED
21,penguin,PENGUIN RANDOM HOUSE LIMITED,PENGUIN BOOKS LIMITED


The model is getting confused by other registered entities which are part of the same group located at the same postcode.

### Option 3: k Nearest Neighbours

The fuzzy matching is too slow if we don't have any reliable address data to pre-filter on. Let's explore a use case where we need to do fuzzy matching on a larger data set in reasonable time.

kNN is a great algorithm for finding data points which are similar to other data points. The search algorithms it can use (eg k-d trees) can be very efficient, and work well with sparse data, so is a good candidate for this fuzzy matching exercise.

To use kNN, we need to embed the company name strings in a numeric space.
We will do this by using a 3-gram count vectoriser (ie number of appearances of three-character strings in the word).
This will make our representation relatively robust to typos / shortenings / reorderings as only some sequences of characters in the company name need to match the true name to look similar.

We also need to choose a distance/similarity metric - we will use the cosine-similarity (which measures the closeness of the angle between the representation vectors) - this reduces the impact of magnitude (ie long versions of company names would be far away from the short version in euclidean distance)

In [16]:
%%time

# Method to convert strings to a design matrix
vectorizer = CountVectorizer(
    analyzer='char',
    lowercase=False,
    ngram_range=(3, 3),
)

# Number of neighbours to consider in the k-nn
n_neighbors = 10

# Vectorise the source data
X_train = vectorizer.fit_transform(df['CompanyName_clean'])

# Specify the model
nbrs = NearestNeighbors(
    n_neighbors=n_neighbors,
    metric='cosine',
).fit(X_train)

CPU times: user 1min 5s, sys: 2.86 s, total: 1min 8s
Wall time: 1min 9s


As the pre-processing here may be a bit obscure, let's inspect what it looks like:

In [17]:
check = np.random.randint(0, X_train.shape[0], 1)

x = pd.DataFrame(
    X_train[check].toarray().transpose(),
    index=vectorizer.get_feature_names(),
    columns=df.iloc[check]['CompanyName_clean']
)

x[x.max(axis=1) > 0].iloc[:10]

CompanyName_clean,A2B MOTOR SALES
MO,1
SA,1
2B,1
A2B,1
ALE,1
B M,1
LES,1
MOT,1
OR,1
OTO,1


#### Inference algorithm
Define a matching algorithm that applies the kNN model and returns the n best guesses

In [18]:
%%time

def match_companies_knn(test_companies):
    
    df_test_companies = pd.DataFrame({'CompanyName':np.atleast_1d(test_companies)})
    
    df_test_companies['CompanyName_clean'] = clean_company_name(df_test_companies['CompanyName'], ignore_words)
    
    X_test = vectorizer.transform(df_test_companies['CompanyName_clean'])
    
    distances, indices = nbrs.kneighbors(X_test)
    
    for i in range(n_neighbors):
        df_test_companies[f'match.{i}.Distance'] = [x[i] for x in distances]
        df_test_companies[f'match.{i}.Index'] = [x[i] for x in indices]
        df_test_companies[f'match.{i}.CompanyName'] = [df['CompanyName'].iloc[x[i]] for x in indices]
        df_test_companies[f'match.{i}.CompanyName_clean'] = [df['CompanyName_clean'].iloc[x[i]] for x in indices]
        df_test_companies[f'match.{i}.CompanyNumber'] = [df['CompanyNumber'].iloc[x[i]] for x in indices]
    
    return df_test_companies

# test the function
match_companies_knn('hiscox')

CPU times: user 4.37 s, sys: 1.64 s, total: 6.01 s
Wall time: 6.82 s


,CompanyName,CompanyName_clean,match.0.Distance,match.0.Index,match.0.CompanyName,match.0.CompanyName_clean,match.0.CompanyNumber,match.1.Distance,match.1.Index,match.1.CompanyName,match.1.CompanyName_clean,match.1.CompanyNumber,match.2.Distance,match.2.Index,match.2.CompanyName,match.2.CompanyName_clean,match.2.CompanyNumber,match.3.Distance,match.3.Index,match.3.CompanyName,match.3.CompanyName_clean,match.3.CompanyNumber,match.4.Distance,match.4.Index,match.4.CompanyName,match.4.CompanyName_clean,match.4.CompanyNumber,match.5.Distance,match.5.Index,match.5.CompanyName,match.5.CompanyName_clean,match.5.CompanyNumber,match.6.Distance,match.6.Index,match.6.CompanyName,match.6.CompanyName_clean,match.6.CompanyNumber,match.7.Distance,match.7.Index,match.7.CompanyName,match.7.CompanyName_clean,match.7.CompanyNumber,match.8.Distance,match.8.Index,match.8.CompanyName,match.8.CompanyName_clean,match.8.CompanyNumber,match.9.Distance,match.9.Index,match.9.CompanyName,match.9.CompanyName_clean,match.9.CompanyNumber
0,hiscox,HISCOX,0.0,1985336,HISCOX PLC,HISCOX,02837811,0.133975,1985316,HISCO LIMITED,HISCO,05575398,0.244071,1985340,HISCOX SA,HISCOX SA,FC034787,0.25,4355875,TISCOX LIMITED,TISCOX,07722573,0.25,899487,CHISCO LTD,CHISCO,11040520,0.25,1383436,EHISCO LIMITED,EHISCO,11935887,0.25,3845416,SHISCO (UK) LIMITED,SHISCO,11206974,0.25,1226444,DHISCO (UK) LTD.,DHISCO,09158867,0.292893,1985333,HISCOX MGA LTD,HISCOX MGA,07720593,0.292893,1985322,HISCOX ASM LTD.,HISCOX ASM,06272775


In [19]:
%%time
# Use the function across all test cases

results = match_companies_knn(df_test['CompanyName'])

df_test['knn_first_guess'] = results['match.0.CompanyName']

# How many guesses are required to get the correct match?
df_test['knn_guesses_required'] = -1
for i in range(n_neighbors):
    df_test['knn_guesses_required'] = pd.Series(
        np.where(
            (df_test['knn_guesses_required'] == -1) & (results[f'match.{i}.CompanyName'] == df_test['CompanyName_actual']),
            i + 1,
            df_test['knn_guesses_required']
        )
    )
    
    
## Check how we did:
df_test['knn_correct'] = (results['match.0.CompanyName'] == df_test['CompanyName_actual'])

print(f"{df_test['knn_correct'].sum()} out of {len(df_test)} correct")

df_test.pivot_table(
    values='CompanyName',
    index='knn_correct',
    columns='TestType',
    aggfunc='count'
)

26 out of 40 correct
CPU times: user 8.74 s, sys: 5.87 s, total: 14.6 s
Wall time: 17.3 s


TestType,extra words,normal,shortened,typo,white space,word order
knn_correct,,,,,,
False,3.0,NaN,8.0,2.0,NaN,1.0
True,4.0,9.0,NaN,5.0,5.0,3.0


The model is faster - a few seconds with no filtering on postcode - this would have taken hours with the fuzzy matching algorithm (ie if no postcode was available.

It only got 25 out of 40 correct, can we see where it went wrong?

In [20]:
df_test.pivot_table(
    values='CompanyName',
    index='knn_guesses_required',
    columns='TestType',
    aggfunc='count'
)

TestType,extra words,normal,shortened,typo,white space,word order
knn_guesses_required,,,,,,
-1,NaN,NaN,3.0,1.0,NaN,NaN
1,4.0,9.0,NaN,5.0,5.0,3.0
2,3.0,NaN,2.0,1.0,NaN,NaN
3,NaN,NaN,1.0,NaN,NaN,1.0
6,NaN,NaN,1.0,NaN,NaN,NaN
9,NaN,NaN,1.0,NaN,NaN,NaN


The model would have got more right on the subsequent guesses. Only 4 records where the model didn't identify the right entity at all. This could be improved by a similar postcode / address filtering approach to earlier.

Let's look at where things actually went wrong.

In [21]:
results[df_test['knn_guesses_required'] != 1][['CompanyName', 'match.0.CompanyName', 'match.1.CompanyName']]

,CompanyName,match.0.CompanyName,match.1.CompanyName
4,Digital Partners,DIGITAL PARTNERS LIMITED,B DIGITAL PARTNERS LTD
9,Towers Watson,TOWERS WATSON UK LIMITED,TOWERS WATSON LIMITED
13,Hudson & Thames,HUDSON & THAMES LIMITED,HUDSON THAMES LIMITED
15,Thames & Hudson Publishing,THAMES & HUDSON PUBLISHING LIMITED,THAMES & HUDSON LIMITED
18,faber,FABER LIMITED,FABER GROUP LTD
20,random house,THE RANDOM HOUSE GROUP LIMITED,PENGUIN RANDOM HOUSE LIMITED
21,penguin,PENGUIN GROUP LTD,PENGUIN UK LIMITED
23,penguin random house publishing,RANDOM HOUSE PUBLISHING GROUP LIMITED,PENGUIN RANDOM HOUSE LIMITED
24,pengling randomhouse,DOMHOUSE LIMITED,PENGUIN RANDOM HOUSE LIMITED
31,Dee Bee Tyres and MOT,L.P TYRES & MOT LTD,DEE BEE MOT LIMITED


A lot of the errors are understandable - they have found genuine companies with names very similar to our "informal" name. Further information would be required to avoid these errors, for example: taking into account the address. 

### Model deployment
For sake of example, I will build a simple Flask app and API to deploy the kNN model (see `app.py`)

### Summary
- **Simple join**: quick, but relatively unlikely to succeed in many cases
- **recordlinkage**: very slow at this scale if unable to pre-filter the candidate comparisons on eg postcode, but could be effective if comparisons can be limited. Further exploration of comparison metrics required to fine-tune performance
- **k-NN**: much faster than recordlinkage at scale. Could get the accuracy to a similar or higher level by also accounting for address, and could theoretically implement this with a fuzzy-address-match that recordlinkage may not scale to.

### Closing thoughts and possible next steps
- Appropriate cleaning of the company name did a lot of work. There could be more "stop words" to remove, or other common punctuation usages that should be specially treated to refine this step.
- Fuzzy matching using standard algorithms is slow on large datasets, but if we have eg a well populated postcode to pre-filter with we can mitigate this issue.
- k-Nearest Neighbours can provide a significantly faster approach to fuzzy matching. Our model didn't perfom as well as the recordlinkage model, but this is mostly because we didn't account for postcode in the nearest neighbours algorithm yet. By doing so, I would expect to see a much better accuracy AND speed from this model.
- k-Nearest Neighbours could also therefore be used to account for a fuzzy match between address strings (ie if postcode isn't reliable for exact matches), where the recordlinkage package is likely to be prohibitively slow.
- There is an issue of larger companies having multiple entities with similar names registered at the same postcode. We probably need to explore some sort of deduplication of the companies house data set to identify a single representative entity or group of companies.
- Given that the fuzzy matching and knn return a score/distance, we could set a threshold to avoid returning very unconfident matches (better to return a null)
- Some times we may be trying to match on old company name, so we could also try matching to the previous company names in the Companies House data
- If we have any knowledge of company industry (eg the dataset may relate to construction, or capture some industry codes), we could also use the industry codes in Companies House for matching


#### To apply these approaches to a real-world problem, I would:
- Develop a broader, labelled test set, based on representative "informal" company representations from the task at hand. This would enable us to better tailor the model to the data available. It may have to be labelled with manual input.
- Define an accuracy metric to compare models, which may depend on the relative impact of false-positives and false-negatives
- Test the various approaches on a hold-out data set to see which has the best balance of accuracy and speed for the use case
- Explore options for keeping the CH data up to date, eg by automating the collection of extracts or using their paid services (eg API)
- Depending on the use case for the matching, we may need to deploy the matching algorithm to production as eg an API or flask app